In [1]:
! pip install ChromeDriverManager


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import re
import time

In [3]:
#크롬 옵션 지정
options = webdriver.ChromeOptions()
#헤드리스 옵션 지정
options.add_argument('--headless')

In [4]:
#크로링 막는 것을 피하기 위해 에이전트 입력(로봇X)
UserAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
options.add_argument('user-agent=' + UserAgent)

In [5]:
#드라이브 설정 (자동 다운로드 및 대기)
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(5)

driver.get(url='https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=1990') #1990년대 시작


In [6]:

driver.find_element(By.XPATH, '//*[@id="cntt_chart_year"]/div/div[1]/div[1]/span/a/img').click() #차트보기 클릭
time.sleep(2)

#BS
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 'tbody > tr'의 경우 123개 출력이 되므로 분리
song1 = soup.select('tbody > tr.lst50 ') # 1-50
song2 = soup.select('tbody > tr.lst100 ') # 51-100

songIdList_1 = []
songIdList_2 = []

rank_1 = []
rank_2 = []

title_1 = []
title_2 = []

artist_1 = []
artist_2 = []

cnt = 1

for song in song1:
    # 1-3위까지 rank의 class 이름이 rank top
    if cnt < 4:
        rank = song.select_one('span.rank.top').text
        rank_1.append(rank)
        cnt = cnt + 1
    else:
        rank = song.select_one('span.rank').text
        rank_1.append(rank)
    title = song.select_one('div.ellipsis.rank01').text.strip().replace('\n', '') #strip() 양쪽 빈칸 없앰 #replace \n ->''으로 대체
    title_1.append(title)
    artist = song.select_one('div.ellipsis.rank02 > span').text.strip().replace('\n', '') #'div.ellipsis.rank02 > a'의 경우 artist가 없는 경우 에러
    artist_1.append(artist)
    
    #가사 Id 가져오기
    song_Tag = song.select_one('a.btn.btn_icon_detail')
    songIdList_1.append(re.sub(r'[^0-9]', '', str(song_Tag)))

for song in song2:
    rank = song.select_one('span.rank').text
    rank_2.append(rank)
    title = song.select_one('div.ellipsis.rank01').text.strip().replace('\n', '') #strip() 양쪽 빈칸 없앰 #replace \n ->''으로 대체
    title_2.append(title)
    artist = song.select_one('div.ellipsis.rank02 > span').text.strip().replace('\n', '')
    artist_2.append(artist)
    
    #가사 Id 가져오기
    song_Tag = song.select_one('a.btn.btn_icon_detail')
    songIdList_2.append(re.sub(r'[^0-9]', '', str(song_Tag)))


In [7]:
#순위
rankList = rank_1 + rank_2
#제목
titleList = title_1 + title_2
#가수
artistList = artist_1 + artist_2
#가사 
songIdList = songIdList_1 + songIdList_2 #100개
lyrics = []

for id in songIdList:
    try:
        driver.get('https://www.melon.com/song/detail.htm?songId=' + id)
        time.sleep(2)
        lyric = driver.find_element(By.CLASS_NAME, 'lyric')
        lyrics.append(lyric.text.replace('\n', ''))
    except:
        #가사가 없는 겨우
        lyrics.append('')
        continue

In [8]:
#엑셀로 출력

df = pd.DataFrame({'순위': rankList, '제목': titleList, '가수': artistList, '가사': lyrics})

df.to_excel('1990.xlsx', sheet_name = '1990')
